<img src= "https://mar.prod.image.rndtech.de/var/storage/images/haz/nachrichten/wirtschaft/niedersachsen/diese-vw-plattform-soll-autoscout24-und-mobile.de-angreifen/640945376-1-ger-DE/Diese-VW-Plattform-soll-Autoscout24-angreifen_master_reference.jpg">

# [SF_DST-27] Project 6. Car Price prediction 

## Task:
**to create a model that will predict the value of a used car based on its characteristics**

**The training dataset is a sample of data from the www.auto.ru is actual on Aug.24, 2021, which includes all major car models produced in the period from 1990 to 2021 and are selling within a radius of 200 km around Moscow**

In [65]:
import pandas as pd
import numpy as np
import re
import pandas_profiling
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import ExtraTreeRegressor
from tqdm import tqdm
from itertools import combinations
from scipy.stats import ttest_ind
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
pd.options.mode.chained_assignment = None

import os


In [66]:
#pip install -U seaborn
#%pip install -U scikit-learn
#%pip install catboost
#pip install pandas_profiling

In [67]:
RANDOM_SEED = 42
!pip freeze > requirements.txt
VAL_SIZE   = 0.20
CURRENT_DATE = pd.Timestamp.today()

### Functions definition

In [68]:

def check_df_before_merg(d_df1,d_df2):
    ''' the function to compare of data types both datasets
    was copied from SF colleagues'''

    list_of_names1 = list(d_df1.columns)
    temp_dict = {}
    temp_dict['feature_train'] = list_of_names1
    temp_dict['type_train'] = d_df1.dtypes
    temp_dict['sample_train'] = d_df1.loc[5].values
    temp_df1 = pd.DataFrame.from_dict(temp_dict)


    list_of_names2 = list(d_df2.columns)
    temp_dict2 = {}
    temp_dict2['feature_test'] = list_of_names2
    temp_dict2['type_test'] = d_df2.dtypes
    temp_dict2['sample_test'] = d_df2.loc[5].values
    temp_df2 = pd.DataFrame.from_dict(temp_dict2)

    temp_insert = pd.DataFrame(columns=['< - >'])

    temp_df = pd.concat([temp_df1,temp_insert, temp_df2], axis=1, sort=False)
    temp_df.reset_index(inplace = True)
    del temp_df['index']
    temp_df['< - >'] = '| - |'
    display(temp_df)

    temp_dict3 = {}
    temp_df3= pd.DataFrame(temp_df)
    temp_list  = []
    temp_list2  = []
    temp_list3  = []
    temp_list4  = []


    for i in range(len(temp_df)):
        if str(temp_df3['type_train'][i]) != str(temp_df3['type_test'][i]):
            temp_list.append(temp_df3['feature_train'][i])
            temp_list2.append(temp_df3['feature_test'][i])
            temp_list3.append(str(temp_df3['type_train'][i]) + ' != ' + str(temp_df3['type_test'][i]))
            temp_list4.append(i)

    temp_dict3['index']= temp_list4
    temp_dict3['feature_train']= temp_list
    temp_dict3['divergences'] = temp_list3
    temp_dict3['feature_test']= temp_list2

    temp_df4 = pd.DataFrame.from_dict(temp_dict3)
    temp_df4.set_index('index',inplace=True)

    print(f'The data types are different in {len(temp_df4)} columns\n')
    display(temp_df4)

In [69]:
def clean_smb(smb):
    ''' the function does remove the symbols
    in the string data'''

    smb = smb.replace("}",'')
    smb = smb.replace("{",'')
    smb = smb.replace("]",'')
    smb = smb.replace("]",'')
    smb = smb.replace('"','')
    smb = smb.strip('[]')
    smb = smb.split(',')
    return smb

In [70]:
def float_value(feature):
    ''' The function cleans the digits from other 
    symbols and brings them to float'''
    
    pattern = re.compile('([^\d.]+)')
    feature = feature.replace(pattern, '')
    oft_value = feature.value_counts(sort=True).keys()[0]
    feature = feature.replace('', oft_value).astype(float, errors = 'raise')
    return feature

In [71]:
def cat_feature_overview(feature):
    ''' The function outputs the feature's info'''
    
    print(f'Total lines: {len(feature)}')
    print(f'Data types: {feature.dtypes}')
    print(f'Total values: {feature.count()}')
    print(f'The most frequent value: {feature.value_counts(sort=True).keys()[0]}')
    print(f'The less frequent value: {feature.value_counts(sort=True).keys()[-1]}')
    print(f'Unique values: {feature.nunique()}')
    print(f'Missing values: {feature.isna().sum()}')
    sns.countplot(y = feature, order = feature.value_counts(sort=True).index)

    return


In [72]:
def num_feature_overview(feature):
    ''' The function outputs the feature's info'''
    
    print(f'Total lines: {len(feature)}')
    print(f'Data types: {feature.dtypes}')
    print(f'Total values: {feature.count()}')
    print(f'The most frequent value: {feature.value_counts(sort=True).keys()[0]}')
    print(f'The less frequent value: {feature.value_counts(sort=True).keys()[-1]}')
    print(f'Unique values: {feature.nunique()}')
    print(f'Missing values: {feature.isna().sum()}')

    return

In [73]:
def subset_by_iqr(feature, whisker_width=1.5):
    '''Remove outliers from a dataframe by column, including optional
       whiskers, removing rows for which the column value are
       less than Q1-1.5IQR or greater than Q3+1.5IQR.
    '''
    q1 = feature.quantile(0.25)
    q3 = feature.quantile(0.75)
    iqr = q3 - q1
    low_outlier = q1 - whisker_width*iqr
    high_outlier = q3 + whisker_width*iqr
    column_median = feature.median()

    # Apply filter with respect to IQR, including optional whiskers
    # else mode()

    return feature.apply(lambda x: x if (x > low_outlier and x < high_outlier) else column_median)

In [74]:
def check_iqr(feature, whisker_width=1.5):
    '''defines the quantiles at the column, including optional
       whiskers with Q1-1.5IQR and Q3+1.5IQR boarders.
    '''
    q1 = feature.quantile(0.25)
    q3 = feature.quantile(0.75)
    iqr = q3 - q1
    low_outlier = q1 - whisker_width*iqr
    high_outlier = q3 + whisker_width*iqr
    IQRs = pd.DataFrame({'Q1' : [q1], 'Q3' : [q3], 'IQR' : [iqr], 'low_outlier' : [low_outlier], 'high_outlier' : [high_outlier]})
    
    sns.set_style("darkgrid")


    # feature's distribution
    x = feature
    plt.figure(figsize=(10,5))
    sns.distplot(x, kde=True)
    plt.axvline(low_outlier, color='red')
    plt.axvline(high_outlier, color='red')
    plt.axvline(q1, color='blue',linestyle='--')
    plt.axvline(q3, color='blue',linestyle='--')
    plt.show()
    

    return IQRs

In [75]:
def train_brand_pvt(val_body_type, from_year,  to_year, i=0):
    
    model_tbl= train[(train['bodyType'] == val_body_type) & (train['modelDate'].between(from_year, to_year))].pivot_table(values= 'price', index= 'brand', aggfunc= 'std')
    std_n = model_tbl['price']
    low_std = std_n / 1.5
    high_std = std_n * 1.5
    model_tbl['low_std'] = round(low_std,2)
    model_tbl['std_n'] = round(std_n,2)
    model_tbl['high_std'] = round(high_std,2)
    model_tbl.drop(['price'], axis = 1, inplace = True)
    low_std_avg = model_tbl.low_std.mean()
    high_std_avg = model_tbl.high_std.mean()
    model_tbl = model_tbl[(model_tbl['std_n'] >= low_std_avg) & (model_tbl['std_n'] <=  high_std_avg)].reset_index()
    display(model_tbl)
    act_model_lst = model_tbl['brand'].to_list()
    print(f'The most relevant brands sorted by body type "{val_body_type}" in time period from {from_year} to {to_year}','\n', act_model_lst,'\n')
 
    
    return act_model_lst

In [76]:
def metrics_func(y_true, y_pred):
    
    #Mean Absolute Error
    mae = metrics.mean_absolute_error(y_true, y_pred)

    #Mean Absolute Percentage Error
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    
    #Symmetric Mean Absolute Percentage Error
    smape = 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

    #Mean Squared Error*
    mse = metrics.mean_squared_error(y_true, y_pred)

    #Root Mean Squared Error
    rmse = np.sqrt(mse)
    
    #Coefficient of determination
    R_2 = metrics.r2_score(y_true, y_pred)
    

    print(f'MAE Mean Absolute Error {mae}')
    print(f'MAPE  Mean Absolute Percentage Error {mape}%')
    print(f'SMAPE Symmetric Mean Absolute Percentage Error {smape}%')
    print(f'MSE Mean Percentage Error {mse}%')
    print(f'RMSE Root Mean Squared Error {rmse}')
    print(f'R_2 Coefficient of determination {R_2}')
    
    return


# 1. DATA
## 1.1 Import of data

In [77]:
"""
PATH_to_file = '/kaggle/input/sf-dst-car-price-prediction/'
PATH_to_file_data = '/kaggle//input/20210824-auto-ru-train-dataset/'
"""

"\nPATH_to_file = '/kaggle/input/sf-dst-car-price-prediction/'\nPATH_to_file_data = '/kaggle//input/20210824-auto-ru-train-dataset/'\n"

In [78]:
"""
train = pd.read_csv(PATH_to_file_data +'2021-08-29_train.csv')
test = pd.read_csv(PATH_to_file+'test.csv')
submit = pd.read_csv(PATH_to_file+'sample_submission.csv')
"""

"\ntrain = pd.read_csv(PATH_to_file_data +'2021-08-29_train.csv')\ntest = pd.read_csv(PATH_to_file+'test.csv')\nsubmit = pd.read_csv(PATH_to_file+'sample_submission.csv')\n"

In [79]:
train = pd.read_csv('2021-08-29_train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('sample_submission.csv')

In [80]:
pd.set_option('display.max_columns', None)
print('Training dataset shape: ', train.shape)
display(train.head(3))
print('Testing dataset shape: ', test.shape)
display(test.head(3))
print('Submission shape: ', submit.shape)
display(submit.head(2))

Training dataset shape:  (94448, 98)
Testing dataset shape:  (34686, 32)
Submission shape:  (34686, 2)


,Unnamed: 0,brand,color,engineDisplacement,enginePower,mileage,modelDate,model_name,numberOfDoors,productionDate,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,Состояние,sample,12v-socket,abs,airbag-driver,airbag-passenger,airbag-side,alarm,alloy-wheel-disks,ashtray-and-cigarette-lighter,audiosystem-cd,auto-mirrors,automatic-lighting-control,aux,bluetooth,climate-control-2,computer,cruise-control,dark-interior,electro-mirrors,electro-window-back,electro-window-front,esp,fabric-seats,front-centre-armrest,front-seats-heat,immo,isofix,leather,leather-gear-stick,light-cleaner,light-sensor,lock,mirrors-heat,multi-wheel,navigation,park-assist-r,power-child-locks-rear-doors,ptf,rain-sensor,seat-transformation,tinted-glass,usb,wheel-configuration1,wheel-configuration2,wheel-leather,wheel-power,xenon,airbag-curtain,third-rear-headrest,fuelType_бензин,fuelType_газ,fuelType_гибрид,fuelType_дизель,fuelType_электро,fuelType_nan,bodyType_внедорожник 3 дв.,bodyType_внедорожник 5 дв.,bodyType_внедорожник открытый,bodyType_кабриолет,bodyType_компактвэн,bodyType_купе,bodyType_купе-хардтоп,bodyType_лимузин,bodyType_лифтбек,bodyType_микровэн,bodyType_минивэн,bodyType_пикап двойная кабина,bodyType_пикап одинарная кабина,bodyType_пикап полуторная кабина,bodyType_родстер,bodyType_седан,bodyType_седан 2 дв.,bodyType_седан-хардтоп,bodyType_тарга,bodyType_универсал 5 дв.,bodyType_фастбек,bodyType_фургон,bodyType_хэтчбек 3 дв.,bodyType_хэтчбек 4 дв.,bodyType_хэтчбек 5 дв.,bodyType_nan
0,0,SKODA,синий,1.2,105,74000,2013,OCTAVIA,5,2014,роботизированная,EUROPEAN,2,Оригинал,передний,Левый,Не требует ремонта,0,1,1,1,1,1,1,0,1,1,0,0,1,0,0,1,0,1,1,1,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,1,1,1,1,1,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,SKODA,чёрный,1.6,110,60563,2017,OCTAVIA,5,2017,механическая,EUROPEAN,1,Оригинал,передний,Левый,Не требует ремонта,0,0,1,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,0,1,1,0,1,1,1,0,0,0,1,1,1,0,0,1,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,SKODA,серый,1.8,152,88000,2013,SUPERB,5,2014,роботизированная,EUROPEAN,1,Оригинал,передний,Левый,Не требует ремонта,0,2,2,2,2,2,1,1,2,2,1,0,2,0,1,2,1,1,2,2,2,2,1,2,2,2,2,0,1,1,2,2,2,1,0,1,1,2,2,2,1,0,2,2,2,2,1,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,RUB,2014,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,RUB,2017,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,88000,2013,"{""code"":""SUPERB"",""name"":""Superb"",""ru_name"":""Су...",SUPERB,DSG 1.8 AMT (152 л.с.),5,1603226280,RUB,2014,1100658222,"{""id"":""20026323"",""nameplate"":""DSG"",""displaceme...",LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


,sell_id,price
0,1100575026,0
1,1100549428,0



### Features
 nn |Features| Description | Type 
----|---|----|---
0   |bodyType| Car's body type | Categorical
1   |brand | Car's brand | Categorical
2   |color | Color | Categorical
3   |color_hex  | Color hex code| Categorical
4   |complectation_dic| Dict of complectation | Categorical
5   |description | Description by seller | Categorical
6   |engineDisplacement | Displacement of engine | Quantitative
7   |enginePower | Power of engine | Quantitative
8   |equipment_dict | Dict of equipment | Categorical
9   |fuelType | Type of fuel | Categorical
10  |image | Image of the car | Categorical
11  |mileage | Mileage of the car | Quantitative
12  |modelDate | Year of car's model| Ordinal
13  |model_info | Code of car's model | Categorical
14  |model_name | Name of car's model | Categorical
15  |name | Summary data | Categorical
16  |numberOfDoors | Number of doors | Quantitative
17  |parsing_unixtime | Time of parsing | Ordinal
18  |price | Offered price | Quantitative
19  |priceCurrency | Currency | Quantitative
20  |productionDate | Date of production | Ordinal
21  |sell_id | Seller's ID | Ordinal
22  |super_gen | Dict of features | Categorical
23  |vehicleConfiguration | Dict of options | Categorical
24  |vehicleTransmission | Transmission | Categorical
25  |vendor | Vendor | Categorical
26  |Владельцы | Number of owners | Quantitative
27  |Владение | Holding period month | Quantitative
28  |ПТС | Original vehicle registration | Binary
29  |Привод | Gear type| Categorical
30  |Руль | Steering wheel | Categorical
31  |Состояние | Accident-free car | Binary
32  |Таможня | Custom cleared | Binary




## 1.2 Data validation and training dataset matching with test sample before the merging

In [81]:
# comparison of training and test datasets
check_df_before_merg(train, test)

,feature_train,type_train,sample_train,< - >,feature_test,type_test,sample_test
0,Unnamed: 0,int64,5,| - |,NaN,NaN,NaN
1,brand,object,SKODA,| - |,brand,object,SKODA
2,color,object,серый,| - |,color,object,серый
3,engineDisplacement,float64,2.0,| - |,engineDisplacement,object,2.0 LTR
4,enginePower,int64,200,| - |,enginePower,object,200 N12
...,...,...,...,...,...,...,...
109,NaN,NaN,NaN,| - |,sell_id,int64,1100912634
110,NaN,NaN,NaN,| - |,super_gen,object,"{""id"":""5009158"",""displacement"":1984,""engine_ty..."
111,NaN,NaN,NaN,| - |,vehicleConfiguration,object,LIFTBACK ROBOT 2.0
112,NaN,NaN,NaN,| - |,Владение,object,NaN


The data types are different in 101 columns



,feature_train,divergences,feature_test
index,,,
0,Unnamed: 0,int64 != nan,NaN
3,engineDisplacement,float64 != object,engineDisplacement
4,enginePower,int64 != object,enginePower
12,Владельцы,int64 != object,Владельцы
17,sample,int64 != nan,NaN
...,...,...,...
109,NaN,nan != int64,sell_id
110,NaN,nan != object,super_gen
111,NaN,nan != object,vehicleConfiguration


### _For further processing, training and test datasets are being merged._

In [82]:
#the feature Unnamed: 0 isn't in test dataset.
# Unnamed column has no information
train.drop(['Unnamed: 0'], axis = 1, inplace=True)

In [83]:
#in train dataset are not the features 'sell_id','car_url'.
#meaning that they don't have any useful for our model information
#drop these features
test.drop(['sell_id','car_url'], axis = 1, inplace=True)

In [84]:
#to provide same shape of the datasets before merging, the 'price' feature has to be in test feature

test['price'] = float(0)

In [85]:
train['price'] = train.price.astype(float)

AttributeError: 'DataFrame' object has no attribute 'price'


* assign for the testing dataset as 'sample' = 0
* assign for the training dataset as 'sample' = 1

In [ ]:
test['sample'] = 0
train['sample'] = 1
print(f'The training dataset with {train.shape} and testing dataset with {test.shape} can be combined')

In [ ]:
# comparison of training and test datasets
check_df_before_merg(train, test)

In [ ]:
# Merging
df = test.append(train, sort=False).reset_index(drop=True)
print(f'the shape of the merged dataset is {df.shape}')

In [ ]:
df.sample(2)

# 2. Data cleaning and Exploratory Data Analysis (EDA)


## 2.1 Pandas profiling analyse

In [ ]:
pandas_profiling.ProfileReport(df)

#### Profiling summary
* _30 not treated features in the dataset:_
    * _CAT	21_
    * _NUM	6_
    * _UNSUPPORTED	4_ 
    * _BOOL	1_
* _Missing cells (%)	2.2%_
* _Duplicate rows (%)	0.0%_
* _'Состояние','Таможня','priceCurrency',  are constant values and there is no sense for any further treatment._ _They are being droped._
* _'car url' can give no information directly and doesn't matter for the prediction model._
* _'image' can give no information directla and doesn'matter for the prediction model._
* _'super gen' contains a lot of technical information, however the test data set doesn't contain the data in the feature._
* _'sell id' doesn't contain any useful information._
* _'vehicleConfiguration' duplicates the information are provided by other features._
* _'description' doesn't contains useful information._
* _'model info duplicates the information are provided by other features._
* _'equipment_dict' and 'complectation_dict' have a similar data and can be merged._

## 2.2 Features analysis

In [ ]:
df.columns

In [ ]:
# droping the useless features
df.drop(['Состояние', 'Таможня', 'priceCurrency', 'image', 'super_gen',
         'vehicleConfiguration', 'description', 'model_info', 'parsing_unixtime'], axis = 1, inplace=True)

In [ ]:
df.shape

### 2.2.1 bodyType

In [ ]:
#show the feature's info
cat_feature_overview(df.bodyType)

In [ ]:
#filling up by mode
df['bodyType'] = df.bodyType.fillna(df.bodyType.value_counts(sort=True).keys()[0])

In [ ]:
#Dropping the bodyTypes features that are not in the test dataset
test_bodyType_lst = test.bodyType.value_counts().keys()
df = df[(df['bodyType'].isin(test_bodyType_lst))]
df.shape

In [ ]:
# 10 most popular car bodies from test dataset
test_bodies_10 = test.bodyType.value_counts(sort=True).keys()[:10]
test_bodies_10

### 2.2.2 enginePower

In [ ]:
num_feature_overview(df.enginePower)

In [ ]:
df['enginePower'] = df['enginePower'].astype(str)
df['enginePower']  = df['enginePower'].apply(lambda x: x.partition('N12')[0]).astype(int)

In [ ]:
num_feature_overview(df.enginePower)

In [ ]:
df[(df['enginePower']<90)]['brand'].value_counts().plot(kind = 'barh',figsize= (10,7), title='The cars with total power less then 90HP')

In [ ]:
df[(df['enginePower']>600)]['brand'].value_counts().plot(kind = 'barh', figsize= (10,7),title='The cars with total power over 600HP')

In [ ]:
sns.boxplot(df.enginePower, color='blue')

In [ ]:
#Distribution of the enginePower before dropping of the outliers
check_iqr(df.enginePower)

In [ ]:
sns.boxplot(df.enginePower, color='blue')

In [ ]:
#Dropping the lines with the outliers
df.drop(df[df.enginePower > 360].index, inplace=True)
df.shape

In [ ]:
#Distribution of the enginePower after dropping of the outliers
check_iqr(df.enginePower)

### 2.2.3 modelDate

In [ ]:
cat_feature_overview(df.modelDate)

In [ ]:
#check the outliers 
check_iqr(df.modelDate)

In [ ]:
df.modelDate.describe()

In [ ]:
#Sampling the relevant modelDate year's range
df = df[(df.modelDate >= 1990) & (df.modelDate <= 2021)]
df.shape

### 2.2.4 productionDate

In [ ]:
cat_feature_overview(df.productionDate)

In [ ]:
#check the outliers distribution of the productionDate
check_iqr(df.productionDate)

In [ ]:
df.productionDate.describe()

In [ ]:
#Sampling the relevant productionDate year's range
df = df[(df.productionDate >= 1995) & (df.productionDate <= 2021)]
df.shape

In [ ]:
#Distribution of the productionDate after dropping of the outliers
check_iqr(df.productionDate)

In [ ]:
df.shape

### 2.2.5 target feature "price" at the train dataset

In [ ]:
#info from prices of training dataset
num_feature_overview(df.price)

In [ ]:
check_iqr(df[(df['sample'] ==1)].price)

In [ ]:
df[(df['sample'] ==1)].price.describe()

In [ ]:
len(df[(df['price']> 3038000)])

In [ ]:
#dropping the higher outliered data in dataset
df = df[((df['sample']==1) & (df['price']<3038000)) | (df['sample']==0)]

In [ ]:
#For further modeling seems to be used a log-function of price feature
df['price_log']=df['price'].apply(lambda x: np.log(x))

In [ ]:
#check the feature's distribution
check_iqr(df[(df['sample'] ==1)].price_log)

In [ ]:
df.shape

In [ ]:
#The distribution car prices by production years
year = df['productionDate'].values
price = df['price'].values
plt.figure(figsize=(15,10))
plt.scatter(year,price,c='b')

### 2.2.6 brand

In [ ]:
cat_feature_overview(df.brand)

In [ ]:
brand_list = train.brand.unique()
print(f'Total amount of represented brands in dataset is {len(brand_list)}')

#### _Definition of most relevant brands for the model of price prediction_


In [ ]:
brand_lst= []
for mark in test_bodies_10:
    brand_lst += train_brand_pvt(mark, 1990, 2020)

#### _The list of relevant brands_

In [ ]:
relevant_brands_lst = set(brand_lst)
print(relevant_brands_lst,'\n','\n', f'Total amount of relevant brands was reduced to {len(relevant_brands_lst)}')


#### _The updated dataset by relevant models list_

In [ ]:
df = df[df['brand'].isin(relevant_brands_lst)]
df.shape

In [ ]:
cat_feature_overview(df.brand)

### 2.2.7 fuelType

In [ ]:
cat_feature_overview(df.fuelType)

In [ ]:
#check wether are same the fuel types at the test dataset and training dataset

print('at the test dataset', '\n',test.fuelType.value_counts(), '\n')
print('at the train dataset','\n',train.fuelType.value_counts(),'\n')

### 2.2.8 engineDisplacement

In [ ]:
num_feature_overview(df.engineDisplacement)

In [ ]:
df.engineDisplacement.value_counts()

In [ ]:
def float_value(feature):
    '''the cleaning function returns the 
    float value's type
    '''
    pattern = re.compile('([^\d.]+)')
    feature = feature.replace(pattern, '')
    oft_value = feature.value_counts(sort=True).keys()[0]
    feature = feature.replace('', oft_value).astype(float, errors = 'raise')
    return feature

In [ ]:
df['engineDisplacement'] = float_value(df['engineDisplacement'])

In [ ]:
df.engineDisplacement.describe()

In [ ]:
#sampling the relevant engineDisplacement range
df = df[(df.engineDisplacement >= 0.9) & (df.engineDisplacement <= 6)]
df.shape

In [ ]:
#Distribution of the engineDisplacement 
check_iqr(df.engineDisplacement)

In [ ]:
#For further modeling seems to be used a log-function of engineDisplacement  feature
df['engineDisplacement _log']=df['engineDisplacement'].apply(lambda x: np.log(x))

In [ ]:
num_feature_overview(df.engineDisplacement)

### 2.2.9 numberOfDoors

In [ ]:
num_feature_overview(df.numberOfDoors)

In [ ]:
#check the '0' value
df[(df.numberOfDoors == 0)]

In [ ]:
#check
plt.hist(df.numberOfDoors)

### 2.2.10 mileage

In [ ]:
num_feature_overview(df.mileage)

In [ ]:
df.mileage.describe()

In [ ]:
check_iqr(df.mileage)

In [ ]:
df[(df['mileage']>384000.0)].brand.value_counts()

In [ ]:
#apply the outlier's high border on dataframe by mileage
df = df[(df['mileage'] < 384000.0)]
df.shape

### 2.2.11 Владельцы

In [ ]:
cat_feature_overview(df.Владельцы)

In [ ]:
df['Владельцы'] = float_value(df['Владельцы']).astype(int)

In [ ]:
sns.countplot(y = df.Владельцы, order = df.Владельцы.value_counts().index)

### _Statistics by the mileage related to amount of owners_

In [ ]:
owner_1 = df[(df['Владельцы']== 1) & (df['mileage']>0)].mileage.describe()
owner_1 

In [ ]:
owner_2 = df[(df['Владельцы']== 2) & (df['mileage']>0)].mileage.describe()
owner_2

In [ ]:
owner_3 = df[(df['Владельцы']== 3) & (df['mileage']>0)].mileage.describe()
owner_3

In [ ]:
owner_4 = df[(df['Владельцы']== 4) & (df['mileage']>0)].mileage.describe()
owner_4

vehicleTransmission### 2.2.12 vehicleTransmission

In [ ]:
cat_feature_overview(df.vehicleTransmission)

### 2.2.13 Привод

In [ ]:
cat_feature_overview(df.Привод)

### 2.2.14 vendor

In [ ]:
cat_feature_overview(df.vendor)

### 2.2.15 ПТС

In [ ]:
cat_feature_overview(df.ПТС)

In [ ]:
#changing the missing value by mode
df.ПТС.fillna(df.ПТС.value_counts().keys()[0], inplace=True)

### 2.2.16 Руль

In [ ]:
cat_feature_overview(df.Руль)

Due low amounts of the right steering wheel, the feature can be dropped


In [ ]:
df.drop(['Руль'], axis=1, inplace=True)

### 2.2.17 name

In [ ]:
df.name.value_counts()[:10]

_The contain information is in other columnes_

_It has to be dropped_

In [ ]:
df.drop(['name'], axis=1, inplace=True)

### 2.2.18 Владение


In [ ]:
df.Владение.isna().sum()

In [ ]:
df.Владение.value_counts()

_the biggest part of the values looks as not valid_  

_it has to be dropped_

In [ ]:
df.drop(['Владение'], axis=1, inplace=True)

### 2.2.19 model_name

In [ ]:
num_feature_overview(df.model_name)

In [ ]:
#through the changing of the strings to upper letter, the amount of unique values is grouped
df['model_name'] = df.model_name.str.upper()

In [ ]:
#most popular model is by each brand
df.groupby(['brand','model_name'])['model_name'].agg(['count']).sort_values(by='count',ascending= False)


### 2.2.20 color

In [ ]:
cat_feature_overview(df.color)

In [ ]:
#most common color by brands
df.groupby(['brand','color'])['color'].agg(['count']).sort_values(by='count',ascending= False)

### 2.2.21  complectation_dict  &  equipment_dict

 _with nested options dictionary_

_the features look same and they have to be mearged_


In [ ]:
print(f'the complectation_dict has {df.complectation_dict.isna().sum()} of missing values')

print(f'the equipment_dict has {df.equipment_dict.isna().sum()} of missing values')

In [ ]:
df['complectation_dict'] = df['complectation_dict'].fillna('No value').astype(str)
df['equipment_dict'] = df['equipment_dict'].fillna('No value').astype(str)

In [ ]:

pat = "'id': '0'"
df['complectation_dict'] = df['complectation_dict'].map(lambda x: x if pat not in x else 'No value')

In [ ]:
#useful data extracting - string partition method.
cmpl_opt = []
for row in df.complectation_dict:
    try:
        line = row.partition('"available_options":')[-1]
        line = line.partition('"vendor_color')[0]
        cmpl_opt.append(line)
    except:
        cmpl_opt.append('No value')

df['complectation_dict'] = cmpl_opt


In [ ]:
df['complectation_dict'] = df['complectation_dict'].apply(clean_smb)

df['complectation_dict']#check

In [ ]:
# extracting the options form equipment_dict
df['equipment_dict'] = df['equipment_dict'].apply(clean_smb)

row_lst = []
for row in df.equipment_dict:
    eq_lst = []
    for item in row:
        if item == item:
            key = item.split(':')[0]
            eq_lst.append(key)
        else:
            eq_lst.append('No value')

    row_lst.append(eq_lst)

df['equipment_dict'] = row_lst

In [ ]:
df['equipment_dict']#check

In [ ]:
#merging the values from columns equipment_dict and equipment_dict
options = []
for i, j in zip(df.complectation_dict, df.equipment_dict):
    options.append(i+j)
df['options'] = options

In [ ]:
df['options']#check

In [ ]:
#To make a list of most used options
df1 = df
df1 = df.explode('options')

In [ ]:

#List of most used options
options_dict = df1.options.value_counts(sort=True)[1:50]
options_lst = options_dict.keys().map(lambda x: x.replace("\'",'').replace(" ", "")).to_list()
options_lst = set(options_lst)


In [ ]:
#are sampled only popular options
row_lst = []
for row in df.options:
    opt_lst = []
    for item in row:
        for opt in options_lst:
            if item == opt:
                opt_lst.append(item)
            else:
                continue
    row_lst.append(opt_lst)

In [ ]:
#to DataFrame
df['options'] = row_lst
df.options.head(5)

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(options_dict.values, options_dict.index, alpha=0.8)
plt.title('Most popular options')
plt.ylabel('Options', fontsize=12)
plt.xlabel('Number of Occurrences', fontsize=12)
plt.show()

In [ ]:
df.drop(['complectation_dict', 'equipment_dict'], axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
#sort by types
cat_cols = ['bodyType', 'brand', 'color','model_name', 'fuelType', 'vehicleTransmission', 'vendor','Привод', 'options']
num_cols= ['engineDisplacement', 'enginePower','mileage', 'modelDate','productionDate', 'Владельцы','numberOfDoors']
bin_cols = ['ПТС','sample']
target_cols = ['price', 'price_log']


In [ ]:
#correlation matrix of train dataset
df_corr = df[(df['sample']== 1)].drop(['sample'], axis=1)
corr = df_corr.corr()
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "8pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Heatmap")\
    .set_precision(2)\
    .set_table_styles(magnify())

the features are highly correlated with each other:
* modelDate, productionDate, mileage, Владельцы, price, 
* enginePower and engineDisplacement 

In [ ]:
df = df.reset_index(drop=True)
df.shape

#### Summary by EDA

Using quantitative techniques and visual methods of representation, EDA looks to tell a story about the existing data based on a broad examination of patterns, trends, outliers, and unexpected results.



# 3. Data preprocessing

## 3.1 Significance of continuous variables by ANOVA F test 

In [ ]:
temp_df = df[df['sample']==1]
imp_num = pd.Series(f_classif(temp_df[num_cols], temp_df['price_log'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh', title='Significance of continuous variables by ANOVA F test for all brands')

## 3.2 Data encoding

In [ ]:
df

### 3.2.1 Categorical Data Encoding

#### 3.2.2.1 Options

In [ ]:
#OneHotEncoding
df1 = pd.get_dummies(df['options'].apply(pd.Series), prefix='', prefix_sep='', dummy_na=True).sum(level=0, axis=1)
#df1 = pd.concat([df, df1.reindex(df1.index)], axis=1)
df1.head(3)

In [ ]:
df1.shape

In [ ]:
df1.drop(['nan'], axis=1, inplace=True)

In [ ]:
cat_cols.remove('options')
cat_cols

In [ ]:
#Lable encoding
labelencoder = LabelEncoder()
for col in cat_cols:
    df1[col] = labelencoder.fit_transform(df[col])

In [ ]:
df1

### 3.2.2 Binary Data Encoding

In [ ]:
#Lable encoding
labelencoder = LabelEncoder()
df1['ПТС'] = labelencoder.fit_transform(df['ПТС'])

In [ ]:
#Adding to df1 the 'sample' column
df1['sample'] = df['sample']

### 3.2.3 Numerical Data Type's columns

In [ ]:
df1.shape

In [ ]:
#Appending the numerical columns into df1
df1[num_cols] = df[num_cols]

In [ ]:
df1[target_cols] = df[target_cols]

In [ ]:
df1.sample(3)

In [ ]:
df1.info()

### Dataset separation on traininig and testing parts by sample mark 

In [ ]:
df1.replace([np.inf, -np.inf], np.nan, inplace=True)

y = df1[(df1['sample']==1)].price
y_lg = df1[(df1['sample']==1)].price_log

X = df1[(df1['sample']==1)].drop(['price','price_log'], axis=1)
X_sub = df1[(df1['sample']==0)]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
X_train.sample(2)

In [ ]:
X_val.sample(2)

# 4. Machine Learning Model

## 4.1 Exploration ML Models

In [ ]:
#Model Random Forest 
model = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

In [ ]:
#Accuracy metrics
metrics_func(y_val, y_pred)

In [ ]:
#Analysis of variables by importance (random forest model)
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X_train.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
df1.sample(5)

In [ ]:
df1['price_log'] = df1.price_log.fillna(0)

In [ ]:
# Linear Regeression
y_train = df1.price_log.values   
X_train = df1.drop(['price','price_log'], axis=1)

lrg = LinearRegression()
lrg.fit(X_train, y_train)
y_pred = lrg.predict(X_val)


In [ ]:
metrics_func(y_val, y_pred)

### _the accurancy at linear Regression metrics has been worsened with compare to Random Forest Model_

In [ ]:
### gradient boosting model
gbr = GradientBoostingRegressor(n_estimators=250)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_val)


In [ ]:
metrics_func(y_val, y_pred)

### _the accurancy at gradient boosting model has been worsened with compare to Random Forest Model also_

In [ ]:
#Bagging Regression model with Random Forest Model
bgr_rf = BaggingRegressor(model, n_estimators=3, n_jobs=-1, random_state=RANDOM_SEED)
bgr_rf.fit(X_train, y_train)
y_pred = bgr_rf.predict(X_val)
metrics_func(y_val, y_pred)


In [ ]:
model = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
metrics_func(y_val, y_pred)

In [ ]:
#Stacking of Random Forest and Bagging
models = [RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1),
         BaggingRegressor(ExtraTreeRegressor(random_state=RANDOM_SEED), random_state=RANDOM_SEED)]

def stacking_model_predict(d_models, d_X_train, d_y_train, d_X_val):
    d_df = pd.DataFrame()
    for model_ in tqdm(d_models):
        model_.fit(d_X_train, d_y_train)
        y_pred = model_.predict(d_X_val)
        d_df[str(model_)[:6]] = np.exp(y_pred)                   
    return d_df

df2 = stacking_model_predict(models, X_train, y_train, X_val)
df2['y_pred']=df2.mean(axis=1)

In [ ]:

y_pred = np.array(df2['y_pred'])
metrics_func(y_val, y_pred)

### _Stacking of Random Forest and Bagging is a best one_

## 4.2 Hyperparameters tuning

In [ ]:
"""

n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1500, num = 50)]
max_features = ['auto', 'sqrt', 'log2']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'bootstrap': bootstrap}

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, 
                                param_distributions = random_grid, 
                                n_iter = 100, 
                                cv = 3, 
                                verbose=2, 
                                random_state=RANDOM_SEED, 
                                n_jobs = -1)
rf_random.fit(X_train, y_train)
"""

# 5.Submission

In [ ]:
X_train.shape


In [ ]:
y_train.shape

In [ ]:
X_val.shape

In [ ]:

models = [RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1),
         BaggingRegressor(ExtraTreeRegressor(random_state=RANDOM_SEED), random_state=RANDOM_SEED)]

def stacking_model_predict(d_models, d_X_train, d_y_train, d_X_test, d_df):
    for model_ in tqdm(d_models):
        model_.fit(d_X_train, d_y_train)
        y_pred = model_.predict(d_X_test)
        d_df[str(model_)[:6]] = np.round(np.exp(y_pred)/10000,2)*10000
    d_df['price'] = d_df.iloc[:,2:].mean(axis=1)    
    return 

stacking_model_predict(models, X_train, y_train, X_val, submit)


In [ ]:
submit

In [ ]:
submit.head(5)

In [ ]:
submit[['id', 'price']].to_csv(f'submission.csv', index=False)